In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [10]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

In [12]:
X = scaler.fit_transform(x)

In [14]:
X.shape

(768, 8)

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state=3)

In [16]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [19]:
model = Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(X_train,y_train,batch_size=32,epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.6531 - accuracy: 0.6042 - val_loss: 0.6726 - val_accuracy: 0.5519
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6050 - accuracy: 0.6564 - val_loss: 0.6388 - val_accuracy: 0.5844
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5720 - accuracy: 0.6954 - val_loss: 0.6135 - val_accuracy: 0.6234
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5460 - accuracy: 0.7215 - val_loss: 0.5964 - val_accuracy: 0.6234
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5276 - accuracy: 0.7378 - val_loss: 0.5828 - val_accuracy: 0.6429
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5127 - accuracy: 0.7541 - val_loss: 0.5695 - val_accuracy: 0.6688
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5002 - accuracy: 0.7704 - val_loss: 0.5616 - val_accuracy: 0.6948
Epoch 8/100


In [21]:
# How to select appropricate optimizer
# No. of nodes in a layer
# How to select no. of layers
# All in one model

In [24]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.5 MB/s eta 0:00:00


In [26]:
import kerastuner as kt

<ipython-input-26-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [37]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer = optimizer, loss="binary_crossentropy", metrics=['accuracy'])

  return model

In [38]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [39]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

In [40]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [41]:
model = tuner.get_best_models(num_models=1)[0]

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4957 - accuracy: 0.7606 - val_loss: 0.5259 - val_accuracy: 0.7468
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4843 - accuracy: 0.7557 - val_loss: 0.5225 - val_accuracy: 0.7273
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4753 - accuracy: 0.7655 - val_loss: 0.5185 - val_accuracy: 0.7143
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4678 - accuracy: 0.7720 - val_loss: 0.5177 - val_accuracy: 0.7143
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4619 - accuracy: 0.7769 - val_loss: 0.5161 - val_accuracy: 0.7143
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4578 - accuracy: 0.7785 - val_loss: 0.5149 - val_accuracy: 0.7143
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4537 - accuracy: 0.7818 - val_loss: 0.5137 - val_accuracy: 0.7208
Epoch 14